In [47]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow

print(tensorflow.__version__)

2.6.0


In [48]:
import glob
import os

# os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' 

# txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당
txt_list = glob.glob(txt_file_path) 

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


In [49]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환 (인덱스, 문장 순)
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Spoken Intro:]
You ever want something 
that you know you shouldn't have 
The more you know you shouldn't have it, 
The more you want it 
And then one day you get it, 
It's so good too 
But it's just like my girl 


In [50]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [51]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

In [52]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2701 2584 ...    0    0    0]
 [   2    7  156 ...    0    0    0]
 [   2   17    7 ...    0    0    0]
 ...
 [   2  311    1 ...    0    0    0]
 [   2  735    5 ...    0    0    0]
 [   2  735    5 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f8e210377c0>


In [53]:
print(tensor[:3, :10])

[[   2 2701 2584    3    0    0    0    0    0    0]
 [   2    7  156   62  199    3    0    0    0    0]
 [   2   17    7   34    7 1518   15   76    3    0]]


In [54]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 12000: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t
16 : s
17 : that
18 : on
19 : of
20 : .
21 : your
22 : m
23 : we
24 : all
25 : like
26 : is
27 : be
28 : for
29 : up
30 : so
31 : with
32 : can
33 : love
34 : know
35 : just
36 : but
37 : don
38 : no
39 : oh
40 : what
41 : got
42 : this
43 : get
44 : ?
45 : they
46 : when
47 : do
48 : she
49 : yeah
50 : now
51 : baby
52 : if
53 : go
54 : re
55 : he
56 : out
57 : was
58 : one
59 : !
60 : down
61 : ll
62 : want
63 : let
64 : see
65 : there
66 : come
67 : cause
68 : say
69 : her
70 : not
71 : at
72 : make
73 : time
74 : from
75 : back
76 : have
77 : are
78 : how
79 : never
80 : girl
81 : as
82 : man
83 : take
84 : right
85 : way
86 : wanna
87 : im
88 : will
89 : ain
90 : need
91 : ve
92 : tell
93 : here
94 : then
95 : u
96 : gonna
97 : who
98 : more
99 : some
100 : where
101 : feel
102 : life
103 : too
104 : could
105 : his
106 : been
107 : said
108 : good
109 : or
110 

752 : higher
753 : mon
754 : looked
755 : joy
756 : favorite
757 : second
758 : cannot
759 : scared
760 : crack
761 : guy
762 : track
763 : stick
764 : paper
765 : fill
766 : brand
767 : meant
768 : sexy
769 : dear
770 : picture
771 : reach
772 : ones
773 : runnin
774 : lovin
775 : walking
776 : shoes
777 : twenty
778 : makin
779 : fucked
780 : breath
781 : outta
782 : prayer
783 : almost
784 : treat
785 : business
786 : sad
787 : knees
788 : cars
789 : keeps
790 : falling
791 : brown
792 : lift
793 : turned
794 : peace
795 : breathe
796 : looks
797 : skin
798 : kinda
799 : naked
800 : paid
801 : supposed
802 : anybody
803 : ur
804 : probably
805 : piece
806 : bobby
807 : fresh
808 : everywhere
809 : outside
810 : mirror
811 : spot
812 : singing
813 : funny
814 : takin
815 : anymore
816 : hoe
817 : yah
818 : ran
819 : giving
820 : callin
821 : few
822 : queen
823 : telling
824 : quick
825 : homie
826 : thinkin
827 : hang
828 : stone
829 : playing
830 : mary
831 : grow
832 : dancing
833

1397 : whether
1398 : jackson
1399 : drug
1400 : position
1401 : fate
1402 : sweat
1403 : sees
1404 : action
1405 : acting
1406 : loneliness
1407 : clap
1408 : excuse
1409 : yup
1410 : thang
1411 : loot
1412 : bones
1413 : fantasy
1414 : evening
1415 : heal
1416 : held
1417 : mornin
1418 : walks
1419 : filled
1420 : weight
1421 : worried
1422 : police
1423 : company
1424 : remind
1425 : ghetto
1426 : judge
1427 : shoulder
1428 : melody
1429 : california
1430 : admit
1431 : nor
1432 : killin
1433 : bam
1434 : fallin
1435 : th
1436 : test
1437 : boat
1438 : gas
1439 : guitar
1440 : glock
1441 : began
1442 : mister
1443 : build
1444 : split
1445 : calls
1446 : date
1447 : stopped
1448 : noise
1449 : drove
1450 : coast
1451 : older
1452 : leader
1453 : sippin
1454 : fashion
1455 : blunts
1456 : silly
1457 : evil
1458 : willing
1459 : bound
1460 : toast
1461 : breathing
1462 : east
1463 : ate
1464 : pockets
1465 : album
1466 : deserve
1467 : dough
1468 : rum
1469 : jolene
1470 : alert
1471 

2113 : fading
2114 : shadow
2115 : twist
2116 : legacy
2117 : note
2118 : mood
2119 : third
2120 : yard
2121 : waves
2122 : march
2123 : cherry
2124 : thy
2125 : rent
2126 : deeper
2127 : flames
2128 : circle
2129 : revolution
2130 : trigger
2131 : pound
2132 : further
2133 : jet
2134 : mercedes
2135 : hater
2136 : simmer
2137 : eatin
2138 : puttin
2139 : temptation
2140 : henry
2141 : jumpman
2142 : biggest
2143 : treasure
2144 : sigh
2145 : moonlight
2146 : trapped
2147 : motion
2148 : cast
2149 : magazine
2150 : hiding
2151 : plays
2152 : mothers
2153 : shade
2154 : direction
2155 : grateful
2156 : lame
2157 : credit
2158 : goodnight
2159 : bother
2160 : amen
2161 : twelve
2162 : nick
2163 : eggs
2164 : indeed
2165 : gentlemen
2166 : nut
2167 : messing
2168 : usually
2169 : grip
2170 : mix
2171 : applause
2172 : palm
2173 : niggaz
2174 : unbelievable
2175 : jewelry
2176 : putting
2177 : delirious
2178 : cindy
2179 : guaranteed
2180 : ayo
2181 : flo
2182 : timbaland
2183 : rainbow
21

2842 : learning
2843 : prepared
2844 : apartment
2845 : struck
2846 : excited
2847 : humble
2848 : sides
2849 : government
2850 : heavens
2851 : drift
2852 : weapon
2853 : abortion
2854 : al
2855 : smart
2856 : scars
2857 : dozen
2858 : reckless
2859 : waters
2860 : rabbit
2861 : rolls
2862 : books
2863 : indian
2864 : woods
2865 : praying
2866 : layin
2867 : giant
2868 : bloody
2869 : mixed
2870 : chris
2871 : sisters
2872 : chips
2873 : concert
2874 : diss
2875 : millions
2876 : killers
2877 : choo
2878 : brag
2879 : droppin
2880 : skip
2881 : hottest
2882 : sets
2883 : offer
2884 : bi
2885 : dew
2886 : housequake
2887 : willin
2888 : roc
2889 : killa
2890 : models
2891 : thai
2892 : honeys
2893 : uhuh
2894 : rammer
2895 : heathen
2896 : powers
2897 : princess
2898 : resist
2899 : photo
2900 : background
2901 : keepin
2902 : jus
2903 : diana
2904 : hunger
2905 : miracles
2906 : martin
2907 : cameras
2908 : gain
2909 : tease
2910 : curse
2911 : revenge
2912 : climbed
2913 : tits
2914 

...

8396 : meditate
8397 : plural
8398 : pussier
8399 : starships
8400 : potions
8401 : kit
8402 : genie
8403 : dutty
8404 : stitch
8405 : possum
8406 : giuseppe
8407 : prior
8408 : chucky
8409 : traumatized
8410 : bulls
8411 : weirdo
8412 : myxomatosis
8413 : romey
8414 : allstate
8415 : kanyes
8416 : turbo
8417 : dickll
8418 : schizoid
8419 : ello
8420 : harvesting
8421 : erased
8422 : unswallow
8423 : lexapro
8424 : ignorance
8425 : mopping
8426 : idiot
8427 : equinox
8428 : marvin
8429 : mediatakeout
8430 : smoothie
8431 : dessert
8432 : katrina
8433 : kweli
8434 : jews
8435 : hed
8436 : coldplay
8437 : itd
8438 : uhuhuh
8439 : onnn
8440 : primetime
8441 : bakin
8442 : modelin
8443 : blessin
8444 : mali
8445 : cassius
8446 : vandross
8447 : imaginary
8448 : wests
8449 : bonus
8450 : cursing
8451 : lynch
8452 : illa
8453 : results
8454 : peeune
8455 : eheheh
8456 : jars
8457 : uhmm
8458 : aaand
8459 : jewish
8460 : fuckery
8461 : birdland
8462 : reap
8463 : illuminates
8464 : glamour
84

9265 : valet
9266 : donuts
9267 : bendin
9268 : biter
9269 : scorch
9270 : tubes
9271 : duffle
9272 : boosie
9273 : ar
9274 : passport
9275 : thanking
9276 : nino
9277 : balled
9278 : bueller
9279 : fanny
9280 : unruly
9281 : compliments
9282 : adopted
9283 : tammy
9284 : bodied
9285 : marl
9286 : wassap
9287 : realization
9288 : accountant
9289 : boulevard
9290 : tweaking
9291 : predictable
9292 : logical
9293 : drooling
9294 : undercover
9295 : oakland
9296 : ovation
9297 : probly
9298 : clapping
9299 : alaska
9300 : cleaners
9301 : smashing
9302 : boostin
9303 : cakes
9304 : sensei
9305 : whisky
9306 : cashin
9307 : urine
9308 : wobble
9309 : glowin
9310 : blackberry
9311 : bloods
9312 : snitch
9313 : bombastic
9314 : palest
9315 : sawed
9316 : brazilian
9317 : nipples
9318 : remy
9319 : sauna
9320 : factories
9321 : lac
9322 : atl
9323 : flossing
9324 : rug
9325 : hitch
9326 : ceo
9327 : dank
9328 : threaten
9329 : fahrenheit
9330 : links
9331 : beatles
9332 : nutsack
9333 : flips


9952 : adlib
9953 : absence
9954 : abort
9955 : boulder
9956 : shifted
9957 : personally
9958 : ashtray
9959 : launcher
9960 : decades
9961 : dramatic
9962 : somethings
9963 : helpin
9964 : definition
9965 : pineapple
9966 : dopest
9967 : trampled
9968 : becky
9969 : vocals
9970 : erupt
9971 : fucka
9972 : quitting
9973 : trades
9974 : goatee
9975 : refresh
9976 : savor
9977 : sturdy
9978 : jamie
9979 : bland
9980 : del
9981 : rey
9982 : oven
9983 : fury
9984 : drips
9985 : axel
9986 : swallowin
9987 : scrunch
9988 : panickin
9989 : falcon
9990 : insult
9991 : cereal
9992 : gether
9993 : shun
9994 : yourselves
9995 : touring
9996 : facilities
9997 : fingernails
9998 : armrest
9999 : chrysler
10000 : stern
10001 : heeeey
10002 : haunts
10003 : constellations
10004 : glimmering
10005 : sinatra
10006 : flaming
10007 : contemplating
10008 : income
10009 : chunk
10010 : rebuild
10011 : callers
10012 : fluffy
10013 : morbid
10014 : cypress
10015 : jo
10016 : uninvited
10017 : kidney
10018 : 

10841 : fittin
10842 : swv
10843 : telepathy
10844 : chante
10845 : rupaul
10846 : xscape
10847 : zhane
10848 : houson
10849 : scarry
10850 : backshots
10851 : flashbacks
10852 : braxton
10853 : stripping
10854 : resistant
10855 : itchy
10856 : bangles
10857 : bamboo
10858 : sadat
10859 : brainstorm
10860 : beatdown
10861 : treys
10862 : mamis
10863 : bigs
10864 : remarkable
10865 : methtical
10866 : cliffs
10867 : infrareds
10868 : fatter
10869 : ninetythree
10870 : leer
10871 : clot
10872 : tarantino
10873 : incidents
10874 : tamika
10875 : lorraine
10876 : livest
10877 : paneling
10878 : crutches
10879 : dutches
10880 : appetit
10881 : forte
10882 : demented
10883 : scented
10884 : civilized
10885 : pelvis
10886 : suplex
10887 : repetitive
10888 : friction
10889 : avenues
10890 : pagers
10891 : specialize
10892 : scrolls
10893 : twitchin
10894 : beehive
10895 : edward
10896 : psc
10897 : collaboration
10898 : suhen
10899 : bidness
10900 : lexuses
10901 : dominican
10902 : rosewood
1

11669 : fred
11670 : mp
11671 : nursing
11672 : nurses
11673 : burger
11674 : pager
11675 : rawkus
11676 : denver
11677 : rescued
11678 : lousy
11679 : suffocates
11680 : intentionally
11681 : explodin
11682 : smartest
11683 : syndrome
11684 : peephole
11685 : cocksuckers
11686 : monopoly
11687 : quitter
11688 : subliminal
11689 : tupac
11690 : reserved
11691 : phased
11692 : ammo
11693 : anthrax
11694 : popsicle
11695 : adored
11696 : flunk
11697 : stapled
11698 : officer
11699 : tempting
11700 : ashley
11701 : zipper
11702 : computers
11703 : squirting
11704 : threesome
11705 : relieve
11706 : decency
11707 : sync
11708 : lifting
11709 : index
11710 : bullied
11711 : caucasians
11712 : buzzed
11713 : dentist
11714 : oral
11715 : peas
11716 : sprinkled
11717 : crayon
11718 : zebra
11719 : tastin
11720 : mart
11721 : acquainted
11722 : wobblin
11723 : idolize
11724 : beaver
11725 : rave
11726 : worker
11727 : exploding
11728 : flaring
11729 : notebook
11730 : myspace
11731 : stadiums
1

In [55]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2 2701 2584    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[2701 2584    3    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [56]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 12000개로 선언했기 때문에, tokenizer.num_words의 값은 12000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

In [57]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [58]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 12001), dtype=float32, numpy=
array([[[ 1.28859057e-04,  1.95367000e-04, -1.31852416e-04, ...,
         -2.10422557e-04, -3.95941170e-05,  5.81574022e-06],
        [ 4.78859110e-05,  1.97472677e-04, -1.92582171e-04, ...,
         -4.96448309e-04, -3.88690823e-05, -2.47932276e-05],
        [ 2.05520446e-05,  1.55609858e-04, -2.87302508e-04, ...,
         -7.48631952e-04, -3.32821248e-04, -2.27244353e-04],
        ...,
        [ 4.48731153e-04,  4.30959015e-04, -4.57635004e-04, ...,
          4.98222071e-04, -1.38047140e-03,  3.11866141e-04],
        [ 7.97334709e-04,  5.11043996e-04, -8.23561568e-04, ...,
          3.75584845e-04, -1.21476606e-03,  5.37582615e-04],
        [ 1.11251685e-03,  5.80805819e-04, -1.20875111e-03, ...,
          2.66014365e-04, -1.03556609e-03,  7.48594641e-04]],

       [[ 1.28859057e-04,  1.95367000e-04, -1.31852416e-04, ...,
         -2.10422557e-04, -3.95941170e-05,  5.81574022e-06],
        [ 1.65889971e-04,  3.37930076e-04,  6

In [59]:
# 모델의 구조를 확인합니다.
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_2 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [60]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

# Adam 알고리즘을 구현하는 optimzier이며 어떤 optimzier를 써야할지 모른다면 Adam을 쓰는 것도 방법이다.
# 우리가 학습을 할 때 최대한 틀리지 않는 방향으로 학습을 해야한다.
# 여기서 얼마나 틀리는지(loss)를 알게하는 함수가 손실함수 이다.
# 이 손실함수의 최소값을 찾는 것을 학습의 목표로 하며 여기서 최소값을 찾아가는 과정을 optimization 이라하고
# 이를 수행하는 알고리즘을 optimizer(최적화)라고 한다.

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=30) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/30
686/686 [==============================] - 153s 220ms/step - loss: 2.8549
Epoch 2/30
686/686 [==============================] - 153s 223ms/step - loss: 2.4514
Epoch 3/30
686/686 [==============================] - 153s 223ms/step - loss: 2.3159
Epoch 4/30
686/686 [==============================] - 153s 223ms/step - loss: 2.2127
Epoch 5/30
686/686 [==============================] - 153s 223ms/step - loss: 2.1225
Epoch 6/30
686/686 [==============================] - 153s 223ms/step - loss: 2.0400
Epoch 7/30
686/686 [==============================] - 154s 224ms/step - loss: 1.9638
Epoch 8/30
686/686 [==============================] - 154s 224ms/step - loss: 1.8909
Epoch 9/30
686/686 [==============================] - 154s 224ms/step - loss: 1.8216
Epoch 10/30
686/686 [==============================] - 154s 225ms/step - loss: 1.7557
Epoch 11/30
686/686 [==============================] - 155s 225ms/step - loss: 1.6926
Epoch 12/30
686/686 [==============================] - 153s 223

In [61]:
#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.

In [62]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [68]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love it when you call me big poppa <end> '